# Exercise 1

## Imports

In [56]:
import pickle
import os.path
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, ArrayType, FloatType, DoubleType
from itertools import combinations
from typing import Iterable, Any, List, Set

import pandas as pd
import numpy as np
from typing import Union

import matplotlib.pyplot as plt

## Spark initialization

In [57]:
spark = SparkSession.builder \
    .appName('exercise1') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", "16")

## Prepare the data

In [58]:
tracks_df = (spark.read
    .option("multiline", "true")
    .option("quote", '"')
    .option("escape", '"')
    .csv('data/tracks.csv')
)

# rename columns with row values from first row to second row
column_categories = list(zip(*tracks_df.take(2)))
columns = tracks_df.columns
tracks_df = tracks_df.select(F.col(columns[0]).alias('track_id'),
    *(F.col(column).alias("-".join(map(str, categories)))
    for column, categories in zip(columns[1:], column_categories[1:]))
)

tracks_df = (tracks_df
    .filter(F.col("track_id").isNotNull()) 
    .filter(F.col("track_id") != "track_id")
)

tracks_df.show()

+--------+--------------+-------------------+-------------------+--------------+---------------+--------+--------------------+-------------+--------------------+----------+--------------------+------------+----------------+------------------------+----------------------+------------------------+--------------------+---------------+-------------------+----------------+---------+---------------+------------------+----------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------------------+---------+----------+--------------+--------------+--------------+-------------------+-------------------+--------------+---------------+---------------+------------+-----------------+--------------------+--------------+-------------------+--------------------+-------------+--------------+------------+---------------+----------+--------------------+
|track_id|album-comments| album-date_created|album-date_released|album-engineer|album-

In [59]:
# TODO: feature selection? not needed necessarily but...
features_df = (spark.read
    .csv('data/features.csv')
)

# rename columns with row values from first row to second row
column_categories = list(zip(*features_df.take(3)))
columns = features_df.columns
# TODO: DoubleType instead of FloatType?
features_df = features_df.select(F.col(columns[0]).alias('track_id'),
    *(F.col(column).cast(DoubleType()).alias("-".join(map(str, categories)))
    for column, categories in zip(columns[1:], column_categories[1:]))
)

features_df = (features_df
    .filter(F.col("track_id") != "feature")
    .filter(F.col("track_id") != "statistics")
    .filter(F.col("track_id") != "number")
    .filter(F.col("track_id") != "track_id")
)

features_music_columns = features_df.columns.copy()
features_music_columns.remove('track_id')

## Agglomerative clustering (in-memory)

In [60]:
dataset_subset_name = "small"

small_tracks_df = tracks_df.filter(F.col("set-subset") == dataset_subset_name)
small_features_df = (features_df
    .join(small_tracks_df, "track_id", "left")
    .filter(F.col("set-subset").isNotNull())
    .select(features_df.columns)
)

small_features_pd = small_features_df.toPandas()

In [61]:
# calculate the metrics (radius, diameter, density_r, density_d) for each cluster
def calculate_metrics(pd_df, centroids):
    
    cluster = pd_df["cluster"].values[0]
    metrics = pd.DataFrame({'radius': [0], 'diameter': [0],'density_r': [0],'density_d': [0]}, columns=['radius', 'diameter','density_r','density_d'])
    centroid = centroids[cluster].reshape(1,-1)

    matrix = pd_df.drop(columns=["cluster", "track_id"]).to_numpy()
    
    matrix_radius = np.sqrt(np.sum((matrix - centroid)**2, axis=1))
    metrics.loc[0,'radius'] = np.max(matrix_radius)
    # calculate density with radius
    metrics.loc[0,'density_r'] = len(pd_df) / metrics.loc[0,'radius']**2

    for i in range(matrix.shape[0]):
        matrix_diameter = np.sqrt(np.sum((matrix[i:,:] - matrix[i,:])**2, axis=1))

        max_diameter = np.max(matrix_diameter)
        if max_diameter > metrics.loc[0,'diameter']:
            metrics.loc[0,'diameter'] = max_diameter
        
    # calculate density with diameter
    metrics.loc[0,'density_d'] = len(pd_df) / metrics.loc[0,'diameter']**2

    return metrics

In [62]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
import numpy.typing as npt

metrics_pd_array = []

small_features_pd["cluster"] = np.zeros((len(small_features_pd), 1), dtype=np.int32)

def cluster_agglomeratively(data: pd.DataFrame, n_clusters: int) -> npt.NDArray[np.float32]:
    if "cluster" not in data.columns:
        raise ValueError("The 'cluster' column should be present in the dataframe!")
    
    data_features_only = data.drop(columns=["cluster", "track_id"])

    clusterer = AgglomerativeClustering(n_clusters=n_clusters)
    clusterer.fit(data_features_only)
    
    centroid_calculator = NearestCentroid()
    centroid_calculator.fit(data_features_only, clusterer.labels_)

    data["cluster"] = clusterer.labels_
    return centroid_calculator.centroids_

if os.path.exists("./results/metrics_pd_array_pickle.pkl"):
    with open("./results/metrics_pd_array_pickle.pkl",'rb') as f:
        metrics_pd_array = pickle.load(f)

else:
    # i = 8 until 16
    for n_clusters in range(8, 17):
        print(f"n_clusters: {n_clusters}")
        centroids = cluster_agglomeratively(small_features_pd, n_clusters)
        metrics_pd_array.append(small_features_pd.groupby("cluster").apply(calculate_metrics, centroids))

    # TODO: weird column at 0s without name??
    with open("./results/metrics_pd_array_pickle.pkl",'wb') as f:
        pickle.dump(metrics_pd_array, f)

/tmp/ipykernel_4186/1666919071.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  small_features_pd["cluster"] = np.zeros((len(small_features_pd), 1), dtype=np.int32)


In [63]:
# TODO: justify N-clusters choice, maybe create a colored matrix?
for i in metrics_pd_array:
    density_r_average = i["density_r"].mean()
    density_d_average = i["density_d"].mean()
    density_r_variance = i["density_r"].var()
    density_d_variance = i["density_d"].var()

    print(f"Average density_r: {density_r_average}, Average density_d: {density_d_average}, Variance density_r: {density_r_variance}, Variance density_d: {density_d_variance}")
    print("\n")

Average density_r: 7.962116015409432e-05, Average density_d: 3.5993421010529105e-05, Variance density_r: 7.554301594511152e-09, Variance density_d: 1.6325448959405526e-09


Average density_r: 8.468280174708501e-05, Average density_d: 3.664884862339534e-05, Variance density_r: 7.747077822050173e-09, Variance density_d: 1.6211166463782024e-09


Average density_r: 8.164135739606747e-05, Average density_d: 3.464754500675633e-05, Variance density_r: 7.052543814624157e-09, Variance density_d: 1.4927141165628917e-09


Average density_r: 7.530015533838525e-05, Average density_d: 3.330844178763551e-05, Variance density_r: 5.651489897428292e-09, Variance density_d: 1.2235775469777811e-09


Average density_r: 7.083319800111053e-05, Average density_d: 3.12437683959014e-05, Variance density_r: 5.38916128542342e-09, Variance density_d: 1.1658460841988244e-09


Average density_r: 6.755265585371375e-05, Average density_d: 2.9803173044205747e-05, Variance density_r: 5.035945257788826e-09, Variance dens

## BFR Algorithm

In [64]:
# TODO: results can be: density of clusters, number of nodes in each cluster, etc., but not strictly necessary

### Parameters

In [65]:
from sklearn.cluster import DBSCAN

n_clusters = 9

max_memory_used_bytes = int(.1e9)
# Assumes all columns are doubles/longs, and so therefore 8 bytes
rows_per_iteration = max_memory_used_bytes // (8 * len(features_df.columns))

seed_random = 0

# Threshold in terms of standard deviations away from centroid, in each dimension
cluster_distance_threshold_standard_deviations = 1
cluster_distance_threshold = ((cluster_distance_threshold_standard_deviations**2) * len(features_music_columns)) ** 0.5

# TODO: choose threshold and justify decision
compression_set_merge_variance_threshold = float('-inf')

# TODO: choose threshold and justify decision
#? distance threshold for inserting point in a compressed set???
dbscan_eps = 1000

### Initialize clusters

In [66]:
k_centroids = cluster_agglomeratively(small_features_pd, n_clusters)

### Loop

In [67]:
import dataclasses

@dataclasses.dataclass(init=False)
class SummarizedCluster:
    n:      int                     
    sum_:   npt.NDArray[np.float64]
    sumsq_: npt.NDArray[np.float64]
    id_:    Union[int, None]
    tracks: Set[int]

    def __init__(self, dimensions: int, id_: int=None):
        self.n = 0
        self.sum_ = np.zeros((dimensions,), dtype=np.float64)
        self.sumsq_ = np.zeros((dimensions,), dtype=np.float64)
        self.id_ = id_
        self.tracks = set()
    
    def summarize(self, point: npt.NDArray[np.float64], track_id: int):
        self.n += 1
        self.sum_ += point
        self.sumsq_ += point**2
        self.tracks.add(track_id)
    
    def summarize_points(self, points: npt.NDArray[np.float64], track_ids: Set[int]):
        self.n += points.shape[0]
        self.sum_ += np.sum(points, axis=0)
        self.sumsq_ += np.sum(points**2, axis=0)
        self.tracks |= track_ids
        self.test = points

    def centroid(self) -> npt.NDArray[np.float64]:
        return self.sum_ / self.n

    def variance(self) -> npt.NDArray[np.float64]:
        return (self.sumsq_ / self.n) - (self.sum_ / self.n)**2

    def standard_deviation(self) -> npt.NDArray[np.float64]:
        return np.sqrt(self.variance())

    def __add__(self, other) -> 'SummarizedCluster':
        if self.id_ is not None and self.other is not None and self.id_ != self.other:
            raise ValueError(f"Clusters {self} and {other} have different explicit ids ({self.id_} != {other.id_}).")
        res = SummarizedCluster(self.sum_.size, self.id_ if self.id_ is not None else other.id_)
        res.n = self.n + other.n
        res.sum_ = self.sum_ + other.sum_
        res.sumsq_ = self.sumsq_ + other.sumsq_
        res.tracks = self.tracks | other.tracks
        return res

In [68]:
discard_sets: List[SummarizedCluster] = [SummarizedCluster(len(features_music_columns), id_) for id_ in range(n_clusters)]
compression_sets: List[SummarizedCluster] = []
retained_set: pd.DataFrame = pd.DataFrame(data=[], columns=features_music_columns)

Summarize clusters with the clustering of the small dataset subset.

In [69]:
def summarize_cluster_df(cluster_df: pd.DataFrame) -> None:
    cluster_id = cluster_df["cluster"].values[0]
    cluster_features_mtx = cluster_df.drop(columns=["cluster", "track_id"]).to_numpy()
    
    track_ids = set(cluster_df["track_id"].values)

    discard_set = discard_sets[cluster_id]
    discard_set.summarize_points(cluster_features_mtx, track_ids)

small_features_pd.groupby("cluster").apply(summarize_cluster_df)

""


Setup functions

In [70]:
assert len(k_centroids) == n_clusters, "The number of clusters does not coincide with the number of random centroids!"

def mahalanobis_distance_pd(x: pd.DataFrame, s: SummarizedCluster) -> pd.Series:
    return (((x - s.centroid()) / s.standard_deviation())**2).sum(axis=1) ** 0.5

In [71]:
features_without_small_df = (features_df
    .join(small_tracks_df, "track_id", "left")
    .filter(F.col("set-subset").isNull())
    .select(features_df.columns)
)

total_rows = features_without_small_df.count()
split_weights = [1.0] * (1 + (total_rows // rows_per_iteration))

split_dfs = features_without_small_df.randomSplit(split_weights, seed=seed_random)

assign discard set id to each point in a set of loaded points

In [72]:
def assign_discard_sets(loaded_points_pd: pd.DataFrame):
    loaded_points_pd = pd.concat(
        objs=[loaded_points_pd, pd.DataFrame(
            data=np.zeros((loaded_points_pd.shape[0], len(cluster_distance_columns) + 2)),
            columns=cluster_distance_columns + ['min_cluster_distance', 'cluster_id'])],
        axis=1
    )

    for i, discard_set in enumerate(discard_sets):
        loaded_points_pd[f"cluster_distance_{i}"] = mahalanobis_distance_pd(loaded_points_pd.iloc[:, 1:519], discard_set)
    loaded_points_pd["min_cluster_distance"] = loaded_points_pd[cluster_distance_columns].min(axis=1)
    loaded_points_pd["cluster_id"] = loaded_points_pd[cluster_distance_columns].idxmin(axis=1).str.slice(start=len(prefix)).astype(np.int32)

    loaded_points_pd.drop(columns=cluster_distance_columns, inplace=True)

    # Don't consider the points that surpass the threshold for the discard sets
    loaded_points_pd.loc[loaded_points_pd["min_cluster_distance"] >= cluster_distance_threshold, "cluster_id"] = -1
    loaded_points_pd.drop(columns=["min_cluster_distance"], inplace=True)

    return loaded_points_pd

In [73]:
prefix = "cluster_distance_"
cluster_distance_columns = [f"{prefix}{i}" for i in range(n_clusters)]

def print_progress(progress: float, message: str):
    print(f"[{progress:3%}] {message:100}", end="\r")

print_progress(0, "Initialized BFR")

# TODO: parallelize with Spark (groupyby + apply), arranjar forma de poderem mexer no mesmo discard set (por exemplo cada um ter a sua cópia e depois juntar tudo, (em termos de resultados pode vir a ser pior ????)
for split_idx, loaded_points_df in enumerate(split_dfs):
    progress = split_idx / len(split_weights)
    
    print_progress(progress, "Collecting split into memory...")
    loaded_points_pd = loaded_points_df.toPandas()

    print_progress(progress, "Clustering with the Mahalanobis distance...")
    
    loaded_points_pd = assign_discard_sets(loaded_points_pd)
    
    print_progress(progress, "Calculated and collected Mahalanobis distances")

    for cluster_id, cluster_df in loaded_points_pd.groupby("cluster_id"):
        track_ids = cluster_df["track_id"]
        features_list = cluster_df.iloc[:, 1:519]

        print_progress(progress, f"Evaluating cluster {cluster_id}...")

        # Step 3 - check which points go to the discard sets
        if cluster_id != -1:
            discard_set = discard_sets[cluster_id]
            discard_set.summarize_points(features_list, set(track_ids))
        
        # Step 4 - check which points go to the compression sets or the retained set
        else:
            matrix_to_cluster = pd.concat(objs=[features_list, retained_set], axis=0)

            # Use same distance as above
            clusterer = DBSCAN(eps=dbscan_eps, metric='euclidean')
            clusterer.fit(matrix_to_cluster)

            retained_set = matrix_to_cluster[clusterer.labels_ == -1]

            mini_clusters = set(clusterer.labels_) - {-1}

            # Create compression sets
            compression_sets_temp = [SummarizedCluster(len(features_music_columns), None) for _ in mini_clusters]
            for mini_cluster_id in mini_clusters:
                compression_sets_temp[mini_cluster_id].summarize_points(matrix_to_cluster[clusterer.labels_ == mini_cluster_id], set(track_ids))
            
            compression_sets.extend(compression_sets_temp)
        
    print_progress(progress, f"Finished evaluating clusters (compression sets: {len(compression_sets)}, retained set: {len(retained_set)})")

    # Step 5 - merge compression sets
    compressing = True
    while compressing:
        compressing = False
        merged_compression_sets = []
        compression_set_idxs_to_remove = set()

        for (idx_1, compression_set_1), (idx_2, compression_set_2) in combinations(enumerate(compression_sets), 2):
            if idx_1 in compression_set_idxs_to_remove or idx_2 in compression_set_idxs_to_remove:
                continue

            merged_compression_set = compression_set_1 + compression_set_2
            if (merged_compression_set.variance() < compression_set_merge_variance_threshold).any():
                merged_compression_sets.append(merged_compression_set)
                compression_set_idxs_to_remove.add(idx_1)
                compression_set_idxs_to_remove.add(idx_2)
                compressing = True
        
        compression_sets = [cs for i, cs in enumerate(compression_sets) if i not in compression_set_idxs_to_remove]
        compression_sets.extend(merged_compression_sets)

        print_progress(progress, f"Completed one compression (compression sets: {len(compression_sets)})")
    
    print_progress((split_idx + 1) / len(split_weights), f"Finished one of the splits (compression sets: {len(compression_sets)}, retained set: {len(retained_set)})")

# Step 6 - merge CS and RS into DS (but we won't yet)
print()

[100.000000%] Finished one of the splits (compression sets: 60, retained set: 1762)                               


## BFR cluster visualization

In [74]:
cluster_idxs = list(range(n_clusters))

genres = [row["track-genre_top"] for row in tracks_df.select("track-genre_top").distinct().collect()]

genre_counts_for_each_discard_set = [
    {
        row["track-genre_top"]:row["count"]
        for row in tracks_df
            .filter(F.col("track_id").isin(discard_set.tracks))
            .groupby("track-genre_top")
            .count()
            .collect()
    }
    for discard_set in discard_sets
]

genre_counts = {
    genre:np.array([(genre_counts_for_each_discard_set[discard_set_idx][genre] if genre in genre_counts_for_each_discard_set[discard_set_idx] else 0) for discard_set_idx in range(n_clusters)])
    for genre in genres
}

KeyError: "['Old-Time / Historic', 'Blues', 'Easy Listening'] not in index"

In [ ]:
width = 0.5

fig, ax = plt.subplots(figsize=(9, 7))
bottom = np.zeros(n_clusters)

for genre in genres:
    p = ax.bar(cluster_idxs, genre_counts[genre], width, label=genre, bottom=bottom)
    bottom += genre_counts[genre]

ax.set_title("Number of tracks per genre on each cluster (discard set)")
ax.legend(loc="upper right", fontsize=10)
ax.set_xlabel("Cluster")
ax.set_ylabel("Count")

plt.show()